# Feature Engineering

This notebook walks through:
- Generating trailing features
- Combining and preparing the data for modeling

## Setup

In [24]:
# Import packages
import pandas as pd

# Reload modules in case changes are made to the codebase
%reload_ext autoreload
%autoreload 2

# Import modules
from lib import feature_generator as fg


### Set display preferences
# Tabular
pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',500)

# Warnings
import warnings; warnings.filterwarnings(action='ignore')

## Select Operators

Features can be generated for any subset of operators

In [25]:
operators = ['DE(Amprion)', 'DE(50Hertz)', 'DE(TenneT GER)', 'DE(TransnetBW)', 'FR', 'IT', 'ES']

## Trailing Features

**Given how much variability is observed over relatively short periods of time, features representing generation over the past few hours or days will help algorithms levelset. Quanitfying multi-day and intra-day trends will also be useful**

The following trailing features are generated:
- Current Forecasts
    - Trailing generation for the three periods immediately before the cutoff (45 minutes for 15 minute intervals; 3 hours for hourly intervals)
    - Rate of change in generation over the last three periods
    - Summary statistics (mean, median, min, and max) over the last hour
    - Summary statistics (mean, median, min, and max) over the last 3 hours
- Day ahead forecasts
    - Generation at the same point in time for the last 3 days
    - Rate of change in generation for the same point in time over the last 2 days
    - Summary statistics (mean, median, min, and max) for the same point in time over the last 3 days
    
The sourcing and processing of generation data isn't instant - an initial lag variable is set to 1 hour to accommodate this. Predictions for noon for instance, will only use data up to 11 am. This can be adjusted.

Type `??TrailingFeatureGenerator` for more details

## Feature Generation

The FeatureGenerator class prepares the data for modeling. Trailing features are generated, and actuals, weather, and capacity data is integrated to write a model_data object. Nightime is filtered out. The class can handle both historical data and forecasts. The data is saved in [feather](https://blog.cloudera.com/blog/2016/03/feather-a-fast-on-disk-format-for-data-frames-for-r-and-python-powered-by-apache-arrow/) format for quicker reading and writing. The initial lag variable can be adjusted - it is set at 1 hour by default. Usage is shown below

In [27]:
f = fg.FeatureGenerator(operators, 'hist', horizon=1)
f.generate()
print(f.model_data.shape)
f.model_data.head()

(294697, 281)


,operator,solar,int_start,year,month,week,hour,minute,month_year,solcap,1h_max,1h_mean,1h_median,1h_min,3h_max,3h_mean,3h_median,3h_min,day_bef,day_bef_2,day_bef_3,day_bef_roc,multiday_max,multiday_mean,multiday_median,multiday_min,per_bef,per_bef_2,per_bef_3,per_bef_roc,d_1_apparentTemperatureHigh,d_1_apparentTemperatureHighTime,d_1_apparentTemperatureLow,d_1_apparentTemperatureLowTime,d_1_apparentTemperatureMax,d_1_apparentTemperatureMaxTime,d_1_apparentTemperatureMin,d_1_apparentTemperatureMinTime,d_1_cloudCover,d_1_dewPoint,d_1_humidity,d_1_icon,d_1_moonPhase,d_1_near_st,d_1_precipAccumulation,d_1_precipIntensity,d_1_precipProbability,d_1_precipType,d_1_pressure,d_1_sunrise,d_1_sunset,d_1_temperatureHigh,d_1_temperatureHighTime,d_1_temperatureLow,d_1_temperatureLowTime,d_1_temperatureMax,d_1_temperatureMaxTime,d_1_temperatureMin,d_1_temperatureMinTime,d_1_uvIndex,d_1_uvIndexTime,d_1_visibility,d_1_windBearing,d_1_windSpeed,d_2_apparentTemperatureHigh,d_2_apparentTemperatureHighTime,d_2_apparentTemperatureLow,d_2_apparentTemperatureLowTime,d_2_apparentTemperatureMax,d_2_apparentTemperatureMaxTime,d_2_apparentTemperatureMin,d_2_apparentTemperatureMinTime,d_2_cloudCover,d_2_dewPoint,d_2_humidity,d_2_icon,d_2_moonPhase,d_2_near_st,d_2_precipAccumulation,d_2_precipIntensity,d_2_precipProbability,d_2_precipType,d_2_pressure,d_2_sunrise,d_2_sunset,d_2_temperatureHigh,d_2_temperatureHighTime,d_2_temperatureLow,d_2_temperatureLowTime,d_2_temperatureMax,d_2_temperatureMaxTime,d_2_temperatureMin,d_2_temperatureMinTime,d_2_uvIndex,d_2_uvIndexTime,d_2_visibility,d_2_windBearing,d_2_windSpeed,d_3_apparentTemperatureHigh,d_3_apparentTemperatureHighTime,d_3_apparentTemperatureLow,d_3_apparentTemperatureLowTime,d_3_apparentTemperatureMax,d_3_apparentTemperatureMaxTime,d_3_apparentTemperatureMin,d_3_apparentTemperatureMinTime,d_3_cloudCover,d_3_dewPoint,d_3_humidity,d_3_icon,d_3_moonPhase,d_3_near_st,d_3_precipAccumulation,d_3_precipIntensity,d_3_precipProbability,d_3_precipType,d_3_pressure,d_3_sunrise,d_3_sunset,d_3_temperatureHigh,d_3_temperatureHighTime,d_3_temperatureLow,d_3_temperatureLowTime,d_3_temperatureMax,d_3_temperatureMaxTime,d_3_temperatureMin,d_3_temperatureMinTime,d_3_uvIndex,d_3_uvIndexTime,d_3_visibility,d_3_windBearing,d_3_windSpeed,d_4_apparentTemperatureHigh,d_4_apparentTemperatureHighTime,d_4_apparentTemperatureLow,d_4_apparentTemperatureLowTime,d_4_apparentTemperatureMax,d_4_apparentTemperatureMaxTime,d_4_apparentTemperatureMin,d_4_apparentTemperatureMinTime,d_4_cloudCover,d_4_dewPoint,d_4_humidity,d_4_icon,d_4_moonPhase,d_4_near_st,d_4_precipAccumulation,d_4_precipIntensity,d_4_precipProbability,d_4_precipType,d_4_pressure,d_4_sunrise,d_4_sunset,d_4_temperatureHigh,d_4_temperatureHighTime,d_4_temperatureLow,d_4_temperatureLowTime,d_4_temperatureMax,d_4_temperatureMaxTime,d_4_temperatureMin,d_4_temperatureMinTime,d_4_uvIndex,d_4_uvIndexTime,d_4_visibility,d_4_windBearing,d_4_windSpeed,d_5_apparentTemperatureHigh,d_5_apparentTemperatureHighTime,d_5_apparentTemperatureLow,d_5_apparentTemperatureLowTime,d_5_apparentTemperatureMax,d_5_apparentTemperatureMaxTime,d_5_apparentTemperatureMin,d_5_apparentTemperatureMinTime,d_5_cloudCover,d_5_dewPoint,d_5_humidity,d_5_icon,d_5_moonPhase,d_5_near_st,d_5_precipAccumulation,d_5_precipIntensity,d_5_precipProbability,d_5_precipType,d_5_pressure,d_5_sunrise,d_5_sunset,d_5_temperatureHigh,d_5_temperatureHighTime,d_5_temperatureLow,d_5_temperatureLowTime,d_5_temperatureMax,d_5_temperatureMaxTime,d_5_temperatureMin,d_5_temperatureMinTime,d_5_uvIndex,d_5_uvIndexTime,d_5_visibility,d_5_windBearing,d_5_windSpeed,h_1_apparentTemperature,h_1_cloudCover,h_1_dewPoint,h_1_humidity,h_1_icon,h_1_near_st,h_1_precipAccumulation,h_1_precipIntensity,h_1_precipProbability,h_1_precipType,h_1_pressure,h_1_temperature,h_1_uvIndex,h_1_visibility,h_1_windBearing,h_1_windSpeed,h_2_apparentTemperature,h_2_cloudCover,h_2_dewPoint,h_2_humidity,h_2_icon,h_2_near_st,h_2_precipAccumulation,h_2_precipIntensity,h_

The dataset can also be generated at the hourly level by setting the hourly flag to True

In [29]:
f = fg.FeatureGenerator(operators, 'hist', horizon=1, hourly=True)
f.generate()
print(f.model_data.shape)
f.model_data.head()

(108583, 281)


,operator,int_start,solar,year,month,week,hour,minute,month_year,solcap,1h_max,1h_mean,1h_median,1h_min,3h_max,3h_mean,3h_median,3h_min,day_bef,day_bef_2,day_bef_3,day_bef_roc,multiday_max,multiday_mean,multiday_median,multiday_min,per_bef,per_bef_2,per_bef_3,per_bef_roc,d_1_apparentTemperatureHigh,d_1_apparentTemperatureHighTime,d_1_apparentTemperatureLow,d_1_apparentTemperatureLowTime,d_1_apparentTemperatureMax,d_1_apparentTemperatureMaxTime,d_1_apparentTemperatureMin,d_1_apparentTemperatureMinTime,d_1_cloudCover,d_1_dewPoint,d_1_humidity,d_1_icon,d_1_moonPhase,d_1_near_st,d_1_precipAccumulation,d_1_precipIntensity,d_1_precipProbability,d_1_precipType,d_1_pressure,d_1_sunrise,d_1_sunset,d_1_temperatureHigh,d_1_temperatureHighTime,d_1_temperatureLow,d_1_temperatureLowTime,d_1_temperatureMax,d_1_temperatureMaxTime,d_1_temperatureMin,d_1_temperatureMinTime,d_1_uvIndex,d_1_uvIndexTime,d_1_visibility,d_1_windBearing,d_1_windSpeed,d_2_apparentTemperatureHigh,d_2_apparentTemperatureHighTime,d_2_apparentTemperatureLow,d_2_apparentTemperatureLowTime,d_2_apparentTemperatureMax,d_2_apparentTemperatureMaxTime,d_2_apparentTemperatureMin,d_2_apparentTemperatureMinTime,d_2_cloudCover,d_2_dewPoint,d_2_humidity,d_2_icon,d_2_moonPhase,d_2_near_st,d_2_precipAccumulation,d_2_precipIntensity,d_2_precipProbability,d_2_precipType,d_2_pressure,d_2_sunrise,d_2_sunset,d_2_temperatureHigh,d_2_temperatureHighTime,d_2_temperatureLow,d_2_temperatureLowTime,d_2_temperatureMax,d_2_temperatureMaxTime,d_2_temperatureMin,d_2_temperatureMinTime,d_2_uvIndex,d_2_uvIndexTime,d_2_visibility,d_2_windBearing,d_2_windSpeed,d_3_apparentTemperatureHigh,d_3_apparentTemperatureHighTime,d_3_apparentTemperatureLow,d_3_apparentTemperatureLowTime,d_3_apparentTemperatureMax,d_3_apparentTemperatureMaxTime,d_3_apparentTemperatureMin,d_3_apparentTemperatureMinTime,d_3_cloudCover,d_3_dewPoint,d_3_humidity,d_3_icon,d_3_moonPhase,d_3_near_st,d_3_precipAccumulation,d_3_precipIntensity,d_3_precipProbability,d_3_precipType,d_3_pressure,d_3_sunrise,d_3_sunset,d_3_temperatureHigh,d_3_temperatureHighTime,d_3_temperatureLow,d_3_temperatureLowTime,d_3_temperatureMax,d_3_temperatureMaxTime,d_3_temperatureMin,d_3_temperatureMinTime,d_3_uvIndex,d_3_uvIndexTime,d_3_visibility,d_3_windBearing,d_3_windSpeed,d_4_apparentTemperatureHigh,d_4_apparentTemperatureHighTime,d_4_apparentTemperatureLow,d_4_apparentTemperatureLowTime,d_4_apparentTemperatureMax,d_4_apparentTemperatureMaxTime,d_4_apparentTemperatureMin,d_4_apparentTemperatureMinTime,d_4_cloudCover,d_4_dewPoint,d_4_humidity,d_4_icon,d_4_moonPhase,d_4_near_st,d_4_precipAccumulation,d_4_precipIntensity,d_4_precipProbability,d_4_precipType,d_4_pressure,d_4_sunrise,d_4_sunset,d_4_temperatureHigh,d_4_temperatureHighTime,d_4_temperatureLow,d_4_temperatureLowTime,d_4_temperatureMax,d_4_temperatureMaxTime,d_4_temperatureMin,d_4_temperatureMinTime,d_4_uvIndex,d_4_uvIndexTime,d_4_visibility,d_4_windBearing,d_4_windSpeed,d_5_apparentTemperatureHigh,d_5_apparentTemperatureHighTime,d_5_apparentTemperatureLow,d_5_apparentTemperatureLowTime,d_5_apparentTemperatureMax,d_5_apparentTemperatureMaxTime,d_5_apparentTemperatureMin,d_5_apparentTemperatureMinTime,d_5_cloudCover,d_5_dewPoint,d_5_humidity,d_5_icon,d_5_moonPhase,d_5_near_st,d_5_precipAccumulation,d_5_precipIntensity,d_5_precipProbability,d_5_precipType,d_5_pressure,d_5_sunrise,d_5_sunset,d_5_temperatureHigh,d_5_temperatureHighTime,d_5_temperatureLow,d_5_temperatureLowTime,d_5_temperatureMax,d_5_temperatureMaxTime,d_5_temperatureMin,d_5_temperatureMinTime,d_5_uvIndex,d_5_uvIndexTime,d_5_visibility,d_5_windBearing,d_5_windSpeed,h_1_apparentTemperature,h_1_cloudCover,h_1_dewPoint,h_1_humidity,h_1_icon,h_1_near_st,h_1_precipAccumulation,h_1_precipIntensity,h_1_precipProbability,h_1_precipType,h_1_pressure,h_1_temperature,h_1_uvIndex,h_1_visibility,h_1_windBearing,h_1_windSpeed,h_2_apparentTemperature,h_2_cloudCover,h_2_dewPoint,h_2_humidity,h_2_icon,h_2_near_st,h_2_precipAccumulation,h_2_precipIntensity,h_